# Submission Preparation

In [26]:
import pandas as pd
import os

SYNTH_DATA_PATH = '../data'
OUT_SYNTH_DATA_PATH = '../output'
SOLUTION_FILE = 'solution_28_07_2023_18_29_25.csv'

- We have provided a sample solution.csv file for your reference. Your submission must
follow the same format.
- If you don’t provide values for all valid indices, a default value i.e. zero will be considered
as the value for those indices. This may result into constraint violation.
- For data_type depot_location and refinery_location, entries under destination_index and
value are not required and will be disregarded. You may choose to keep these entries blank.
- For data_type biomass_forecast, entries under destination_index are not required and will be
disregarded. You may choose to keep these entries blank.
- Optimized supply chain infrastructure proposed in your solution must be the same for both
year 2018 and 2019. Consequently, the entries for data_type depot_location and
refinery_location will be agnostic to year. Use year = 20182019 for such common entries.
- You can only place one depot per grid block/location. Similarly, You can only place one
biorefinery per grid block/location.
- Your solution will be eligible for ranking only if it satisfies all the constraints for 2018 and 2019.
- You do not need to submit your source code files. When you submit your solution, you can
ignore the "Upload source code" field.
- We will keep the first year (2018) of your solution for the public leaderboard. You can test
your solution any time and see how it ranks.
- We will keep the second year (2019) of your solution for the private leaderboard and it will
be used to determine the finalists.

In [27]:
df_submission = pd.read_csv(os.path.join(SYNTH_DATA_PATH, 'sample_submission.csv'))
df_submission.head()

,year,data_type,source_index,destination_index,value
0,20182019,depot_location,1256,NaN,NaN
1,20182019,depot_location,1595,NaN,NaN
2,20182019,depot_location,1271,NaN,NaN
3,20182019,depot_location,2001,NaN,NaN
4,20182019,depot_location,2201,NaN,NaN


In [28]:
df_submission['data_type'].value_counts()

data_type
biomass_demand_supply    21646
biomass_forecast          4836
pellet_demand_supply       152
depot_location              21
refinery_location            4
Name: count, dtype: int64

In [29]:
df_sol = pd.read_csv(os.path.join(OUT_SYNTH_DATA_PATH, SOLUTION_FILE))
df_sol.columns = ["data_type", "solution"]
df_sol.head()

,data_type,solution
0,b_2018_809_809,0.0
1,b_2018_809_1278,0.0
2,b_2018_809_1060,0.0
3,b_2018_809_744,0.0
4,b_2018_809_1004,0.0


In [30]:
df_fc = pd.read_csv(os.path.join(SYNTH_DATA_PATH, 'Biomass_History_Synthetic.csv'))
df_fc.head()

,Index,Latitude,Longitude,2018,2019
0,0,24.66818,71.33144,7.387925,5.180296
1,1,24.66818,71.41106,40.431847,42.126945
2,2,24.66818,71.49069,59.181629,73.203232
3,3,24.66818,71.57031,74.536720,101.067352
4,4,24.66818,71.64994,16.531315,26.086885


In [31]:
df_fc_sol = df_fc.copy()
df_fc_sol = df_fc_sol.melt(value_vars=['2018', '2019'], var_name='year', value_name='biomass', id_vars=['Index'])
df_fc_sol['data_type'] = 'biomass_forecast'
df_fc_sol['destination_index'] = None
df_fc_sol = df_fc_sol[['data_type', 'year', 'Index', 'destination_index', 'biomass']]
df_fc_sol.columns = ['data_type', 'year', 'source_index', 'destination_index', 'value']
df_fc_sol.head()

,data_type,year,source_index,destination_index,value
0,biomass_forecast,2018,0,None,7.387925
1,biomass_forecast,2018,1,None,40.431847
2,biomass_forecast,2018,2,None,59.181629
3,biomass_forecast,2018,3,None,74.536720
4,biomass_forecast,2018,4,None,16.531315


## Biomass

In [32]:
df_sol_proc = df_sol.copy()
df_sol_proc['data_type'] = df_sol_proc['data_type'].str.replace('x_', 'x_20182019_').str.replace('r_', 'r_20182019_')
df_sol_proc = df_sol_proc['data_type'].str.split("_", expand=True)
df_sol_proc.columns = ['data_type', 'year', 'source_index', 'destination_index']

df_sol_proc['value'] = df_sol['solution']
df_sol_proc['data_type'] = df_sol_proc['data_type'].map({'b': 'biomass_demand_supply', 
                                                         'p': 'pellet_demand_supply', 
                                                         'x': 'depot_location', 
                                                         'r': 'refinery_location'})
df_sol_proc = df_sol_proc[df_sol_proc['value'] != 0]

df_sol_proc = pd.concat([df_sol_proc, df_fc_sol])
df_sol_proc.head()

,data_type,year,source_index,destination_index,value
15,biomass_demand_supply,2018,809,1063,170.129959
127,biomass_demand_supply,2018,1278,1063,42.676819
239,biomass_demand_supply,2018,1060,1063,196.065643
351,biomass_demand_supply,2018,744,1063,221.294220
463,biomass_demand_supply,2018,1004,1063,184.094040


In [33]:
df_sol_proc[df_sol_proc['data_type'] == 'depot_location'].head()

,data_type,year,source_index,destination_index,value
50191,depot_location,20182019,1063,None,1.0


In [34]:
df_sol_proc[df_sol_proc['data_type'] == 'refinery_location'].head()

,data_type,year,source_index,destination_index,value
50288,refinery_location,20182019,809,None,1.0


In [35]:
df_sol_proc.data_type.value_counts()

data_type
biomass_forecast         4836
biomass_demand_supply     155
pellet_demand_supply        2
depot_location              1
refinery_location           1
Name: count, dtype: int64

In [36]:
df_sol_proc[df_sol_proc['data_type'] == 'biomass_demand_supply']#.head()

,data_type,year,source_index,destination_index,value
15,biomass_demand_supply,2018,809,1063,170.129959
127,biomass_demand_supply,2018,1278,1063,42.676819
239,biomass_demand_supply,2018,1060,1063,196.065643
351,biomass_demand_supply,2018,744,1063,221.294220
463,biomass_demand_supply,2018,1004,1063,184.094040
...,...,...,...,...,...
24543,biomass_demand_supply,2019,221,1063,153.434204
24655,biomass_demand_supply,2019,1020,1063,149.782852
24767,biomass_demand_supply,2019,1028,1063,15.375629
24879,biomass_demand_supply,2019,505,1063,92.133476


In [37]:
df_sol_proc[df_sol_proc['data_type'] == 'biomass_demand_supply'].destination_index.value_counts()#.head()

destination_index
1063    155
Name: count, dtype: int64

In [38]:
df_fc.loc[1020, :]

Index        1020.000000
Latitude       22.814370
Longitude      73.640570
2018           95.019615
2019          149.782852
Name: 1020, dtype: float64

In [39]:
rest_2 = df_sol_proc[df_sol_proc['data_type'] == 'biomass_demand_supply'].groupby(['year', 'destination_index'])['value'].sum()
rest_2

year  destination_index
2018  1063                 13240.184983
2019  1063                 16808.765179
Name: value, dtype: float64

In [40]:
rest_2 = df_sol_proc[df_sol_proc['data_type'] == 'biomass_demand_supply'].groupby(['year', 'source_index'])['value'].sum()
rest_2

year  source_index
2018  1004            184.094040
      1015            219.028778
      1020             95.019615
      1028             13.498848
      103             111.943245
                         ...    
2019  868             205.152802
      882              30.233065
      891              88.030396
      935             176.405548
      985             115.161858
Name: value, Length: 155, dtype: float64

In [41]:
df_fc.iloc[1063, :]

Index        1063.000000
Latitude       22.733770
Longitude      72.685070
2018          247.525650
2019          151.901977
Name: 1063, dtype: float64

In [42]:
df_sol_proc[df_sol_proc['data_type'] == 'pellet_demand_supply'].head()

,data_type,year,source_index,destination_index,value
25103,pellet_demand_supply,2018,1063,809,14491.933938
37647,pellet_demand_supply,2019,1063,809,17953.653824


In [43]:
df_sol_proc[df_sol_proc['data_type'] == 'pellet_demand_supply'].head()

,data_type,year,source_index,destination_index,value
25103,pellet_demand_supply,2018,1063,809,14491.933938
37647,pellet_demand_supply,2019,1063,809,17953.653824


In [44]:
df_sol_proc.to_csv(os.path.join(OUT_SYNTH_DATA_PATH, 'subm_yearly_test_def.csv'), index=False)

In [45]:
df_sol_proc

,data_type,year,source_index,destination_index,value
15,biomass_demand_supply,2018,809,1063,170.129959
127,biomass_demand_supply,2018,1278,1063,42.676819
239,biomass_demand_supply,2018,1060,1063,196.065643
351,biomass_demand_supply,2018,744,1063,221.294220
463,biomass_demand_supply,2018,1004,1063,184.094040
...,...,...,...,...,...
4831,biomass_forecast,2019,2413,None,5.321604
4832,biomass_forecast,2019,2414,None,0.120626
4833,biomass_forecast,2019,2415,None,0.038879
4834,biomass_forecast,2019,2416,None,1.304297
